# Skeleton analysis with Skan

## extracting a skeleton from an image

In [ ]:
from glob import glob
import imageio as iio

files = glob('data/*.tif')
image0 = iio.imread(files[0], format='fei')

We display the images with [Matplotlib](http://matplotlib.org).

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

fig, ax = plt.subplots()
ax.imshow(image0, cmap='gray');

As an aside, we can extract the pixel spacing in meters from the `.meta` attribute of the ImageIO image:

In [ ]:
import numpy as np

spacing = image0.meta['Scan']['PixelHeight']
spacing_nm = spacing * 1e9  # nm per pixel
dim_nm = np.array(image0.shape) / spacing_nm

fig, ax = plt.subplots()
ax.imshow(image0, cmap='gray',
          extent=[0, dim_nm[1], dim_nm[0], 0]);
ax.set_xlabel('x (nm)')
ax.set_ylabel('y (nm)');

In [ ]:
from skan.pre import threshold

smooth_radius = 5 / spacing_nm  # float OK
threshold_radius = int(np.ceil(50 / spacing_nm))
binary0 = threshold(image0, sigma=smooth_radius,
                    radius=threshold_radius)

fig, ax = plt.subplots()
ax.imshow(binary0);

In [ ]:
from skimage import morphology

skeleton0 = morphology.skeletonize(binary0)

In [ ]:
from skan import draw

fig, ax = plt.subplots()
draw.overlay_skeleton_2d(image0, skeleton0, dilate=1, axes=ax);

## measuring skeleton branches

In [ ]:
from skan import skeleton_to_csgraph

pixel_graph, coordinates, degrees = skeleton_to_csgraph(skeleton0)

The pixel graph is a SciPy [CSR matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) in which entry $(i, j)$ is 0 if pixels $i$ and $j$ are not connected, and otherwise is equal to the distance between pixels $i$ and $j$ in the skeleton. This will normally be 1 between adjacent pixels and $\sqrt{2}$ between diagonally adjacent pixels, but in this can be scaled by a `spacing=` keyword argument that sets the scale (and this scale can be different for each image axis). In our case, we know the spacing between pixels, so we can measure our network in physical units instead of pixels:

In [ ]:
pixel_graph0, coordinates0, degrees0 = \
                skeleton_to_csgraph(
                        skeleton0,
                        spacing=spacing_nm
                )

In [ ]:
from skan import _testdata
g0, c0, _ = skeleton_to_csgraph(_testdata.skeleton0)
g1, c1, _ = skeleton_to_csgraph(_testdata.skeleton1)
fig, axes = plt.subplots(1, 2)

draw.overlay_skeleton_networkx(g0, c0, image=_testdata.skeleton0,
                               axis=axes[0])
draw.overlay_skeleton_networkx(g1, c1, image=_testdata.skeleton1,
                               axis=axes[1]);

In [ ]:
draw.overlay_euclidean_skeleton_2d(image0, branch_data,
                                   skeleton_colormap='magma',
                                   skeleton_color_source='branch-distance');

### The skeleton class

In [ ]:
from skan import Skeleton, summarize
cskel = Skeleton(skeleton0, spacing=spacing_nm,
                 source_image=image0)

In [ ]:
from skimage import feature

si = feature.shape_index(image0, sigma=5, mode='reflect')

plt.imshow(si);

In [ ]:
cskel = Skeleton(skeleton0 * si, spacing=spacing_nm,
                 source_image=image0)

draw.overlay_skeleton_2d_class(cskel);

### The branch data summary

In [ ]:
branch_data = summarize(cskel)
branch_data.head()

In [ ]:
branch_data.hist(column='branch-distance', by='branch-type', bins=100);

## 2. Other applications

---

### Neuroscience

![](https://dfzljdn9uc3pi.cloudfront.net/2018/4312/1/fig-1-2x.jpg)

---

### Plant morphology

(See the [PlantCV docs](https://plantcv.readthedocs.io/en/stable/morphology_tutorial/)!)

![](https://plantcv.readthedocs.io/en/stable/img/tutorial_images/morphology/original_img.jpg)

![](https://plantcv.readthedocs.io/en/stable/img/tutorial_images/morphology/cropped_mask_image.jpg)

![](https://plantcv.readthedocs.io/en/stable/img/tutorial_images/morphology/pruned_segmented_img_mask_tutorial.jpg)

### Remote sensing 

In [ ]:
image = io.imread('data/130_Parana_River_Paraguay.jpg')

In [ ]:
plt.imshow(image);

In [ ]:
d_from_blue = np.sum(np.abs(image - [0, 0, 255]), axis=-1)
plt.imshow(d_from_blue < 250);

In [ ]:
from skimage import morphology

river = morphology.remove_small_objects(d_from_blue < 250, 1000)
river_clean = morphology.binary_closing(river)
plt.imshow(river_clean);

In [ ]:
midpoint = morphology.skeletonize(river_clean)

river_skeleton = Skeleton(midpoint, source_image=image)
summarize(river_skeleton).head()

In [ ]:
draw.overlay_skeleton_2d_class(river_skeleton, image_cmap=None,
                               skeleton_color_source='path_lengths',
                               skeleton_colormap='magma');


## More info:

- our [PeerJ paper](https://peerj.com/articles/4312/) (and [please cite it](https://ilovesymposia.com/2019/05/02/why-you-should-cite-open-source-tools/) if you publish using skan!),
- the ["Complete analysis with skan"](complete_analysis.html) page, and
- the [skan-scripts](https://github.com/jni/skan-scripts) repository.